<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 4: West Nile Virus Prediction

---
    Mubina | Wei Hua | Liubin    

## Problem Statement
Our client is the Centers for Disease Control and Prevention (CDC) who are working with Chicago government to reduce the patients whom are affected by the incurable West Nile Virus. Our team are given a chance to work with Department of Public Health to set up a surveillance control system. 

As a data scientist from a consultancy firm, our task is to build a model and make predictions to determine the period and location of the sprays. We will also be conduting a cost-benefit analysis which include the annual cost projections for various levels of pesticide and quantity of the pesticide spraying to achieve the maximum benefit. 

Our primary stakeholders will be the client (CDC) and the secondary stakeholders will be the government of Chicago. The given dataset consist of the following data:

- Main dataset where public health workers in Chicao set up mosquito traps across the city to test for the presence of West Nile virus. 
- Spray data which records the details of their spraying such as location and date in order to reduce the number of mosquitoes in the area. 
- Weather Data which records the condition of the city. It is believed that hot and dry conditions are more favourable for West Nile virus as compared to cold and wet. 
- Map from openstreet map 

The following is the general workflow for this project: 
+ Data Cleaning
+ Exploratory data analysis (EDA)
+ Pre-processing and feature engineering
+ Modelling and evaluation
+ Cost benefit analysis
+ Conclusion and recommendation

The model will then be evaluated by ROC AUC score and recall score. The objective of the model is to get a high  ROC AUC score and recall score.
The model will then be evaluated by ROC AUC score and the objective of the model is to get a high ROC AUC score.

# Part 1: Data Cleaning 

In this notebook, mainly We have performed Data Cleaning all the datasets:
- train data
- test data
- weather data
- spray data

Data Cleaning including: check and handle missing values, check and remove duplicated values, convert inproper data types. We have also attempted to merge Weather data (station number 1 or 2) into train, test data to create an indicator the nearest weather station to the trap location. 

## Contents:
- [1. Import Libraries](#1.-Import-Libraries)
- [2. Load Data](#2.-Load-Data)
- [3. Data Cleaning](#3.-Data-Cleaning)
- [4. Export Cleaned Data](#4.-Export-Cleaned-Data)

## 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timezone
import time
import seaborn as sns
import matplotlib.pyplot as plt

from geopy.distance import geodesic
import geopy

%matplotlib inline
sns.set_style("darkgrid")
sns.set_palette("viridis")

## 2. Load Data

In [2]:
data_path = "../assets/"

train = pd.read_csv(data_path + "train.csv")
weather = pd.read_csv(data_path + "weather.csv")
spray = pd.read_csv(data_path + "spray.csv")
test = pd.read_csv(data_path + "test.csv")

## 3. Data Cleaning

### 3.1 Train & Test Data Inspection & Cleaning

In [3]:
train.shape

(10506, 12)

In [4]:
test.shape

(116293, 11)

In [5]:
train.head(3)

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0


In [6]:
test.head(3)

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy
0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
1,2,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
2,3,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9


In [7]:
train.columns=train.columns.map(lambda x:x.lower())
test.columns=test.columns.map(lambda x:x.lower())

In [8]:
train.columns

Index(['date', 'address', 'species', 'block', 'street', 'trap',
       'addressnumberandstreet', 'latitude', 'longitude', 'addressaccuracy',
       'nummosquitos', 'wnvpresent'],
      dtype='object')

In [9]:
test.columns

Index(['id', 'date', 'address', 'species', 'block', 'street', 'trap',
       'addressnumberandstreet', 'latitude', 'longitude', 'addressaccuracy'],
      dtype='object')

#### Comments:

- These 2 columns are missing in test data- "nummosquitos" and "wnvpresent"
- We need to solve 2 problems by modleing: (1) regression model to predict "nummosquitos", (2) classification model on (1)'s prediction for wnvpresent.

We will show the model on "Part 3-Model and Recommendation".

In [10]:
# check Trap in train data
train_traps=train.trap.unique()
print(train_traps)
print(len(train_traps))

['T002' 'T007' 'T015' 'T045' 'T046' 'T048' 'T049' 'T050' 'T054' 'T086'
 'T091' 'T094' 'T096' 'T129' 'T143' 'T148' 'T153' 'T159' 'T009' 'T011'
 'T016' 'T019' 'T025' 'T028' 'T031' 'T033' 'T089' 'T090' 'T092' 'T135'
 'T141' 'T142' 'T145' 'T146' 'T147' 'T149' 'T150' 'T151' 'T152' 'T154'
 'T158' 'T162' 'T218' 'T220' 'T001' 'T003' 'T006' 'T008' 'T012' 'T034'
 'T037' 'T040' 'T043' 'T047' 'T051' 'T085' 'T088' 'T161' 'T219' 'T013'
 'T014' 'T018' 'T030' 'T084' 'T144' 'T160' 'T005' 'T017' 'T044' 'T095'
 'T004' 'T035' 'T036' 'T039' 'T060' 'T061' 'T062' 'T065' 'T066' 'T067'
 'T069' 'T070' 'T071' 'T073' 'T074' 'T075' 'T076' 'T077' 'T079' 'T080'
 'T081' 'T082' 'T083' 'T114' 'T155' 'T063' 'T115' 'T138' 'T200' 'T206'
 'T209' 'T212' 'T215' 'T107' 'T128' 'T072' 'T078' 'T097' 'T099' 'T100'
 'T102' 'T103' 'T027' 'T156' 'T157' 'T221' 'T900' 'T903' 'T222' 'T223'
 'T225' 'T227' 'T224' 'T226' 'T229' 'T230' 'T228' 'T232' 'T231' 'T235'
 'T233' 'T236' 'T237' 'T238' 'T094B' 'T054C']
136


In [11]:
# check Trap in train data
test_traps=test.trap.unique()
print(test_traps)
print(len(test_traps))

['T002' 'T007' 'T015' 'T045' 'T046' 'T048' 'T049' 'T050' 'T054' 'T086'
 'T091' 'T094' 'T096' 'T129' 'T143' 'T148' 'T153' 'T159' 'T009' 'T011'
 'T016' 'T019' 'T025' 'T028' 'T031' 'T033' 'T089' 'T090' 'T092' 'T135'
 'T141' 'T142' 'T145' 'T146' 'T147' 'T149' 'T150' 'T151' 'T152' 'T154'
 'T158' 'T162' 'T218' 'T220' 'T001' 'T003' 'T006' 'T008' 'T012' 'T034'
 'T037' 'T040' 'T043' 'T047' 'T051' 'T085' 'T088' 'T161' 'T219' 'T013'
 'T014' 'T018' 'T030' 'T084' 'T144' 'T160' 'T005' 'T017' 'T044' 'T095'
 'T004' 'T035' 'T036' 'T039' 'T060' 'T061' 'T062' 'T065' 'T066' 'T067'
 'T069' 'T070' 'T071' 'T073' 'T074' 'T075' 'T076' 'T077' 'T079' 'T080'
 'T081' 'T082' 'T083' 'T114' 'T155' 'T063' 'T115' 'T138' 'T200' 'T206'
 'T209' 'T212' 'T215' 'T107' 'T128' 'T072' 'T078' 'T097' 'T099' 'T100'
 'T102' 'T103' 'T027' 'T156' 'T157' 'T221' 'T900' 'T903' 'T090A' 'T090B'
 'T090C' 'T200A' 'T128A' 'T200B' 'T218A' 'T218C' 'T218B' 'T222' 'T223'
 'T225' 'T227' 'T224' 'T226' 'T229' 'T230' 'T228' 'T231' 'T232' 'T002A'
 'T

#### Comments:
- there are 136 unique traps in train data, while there are more traps in test data (149 traps).

In [12]:
# check 7 Species of Mosquitoes
train['species'].value_counts()

CULEX PIPIENS/RESTUANS    4752
CULEX RESTUANS            2740
CULEX PIPIENS             2699
CULEX TERRITANS            222
CULEX SALINARIUS            86
CULEX TARSALIS               6
CULEX ERRATICUS              1
Name: species, dtype: int64

In [13]:
test['species'].value_counts()

CULEX PIPIENS/RESTUANS    15359
CULEX RESTUANS            14670
CULEX PIPIENS             14521
CULEX SALINARIUS          14355
CULEX TERRITANS           14351
CULEX TARSALIS            14347
UNSPECIFIED CULEX         14345
CULEX ERRATICUS           14345
Name: species, dtype: int64

#### Comments:

- Both train and test data has the same top 3 specises mosquitos: CULEX PIPIENS/RESTUANS, CULEX RESTUANS and CULEX PIPIENS.

In [14]:
train.isnull().sum()

date                      0
address                   0
species                   0
block                     0
street                    0
trap                      0
addressnumberandstreet    0
latitude                  0
longitude                 0
addressaccuracy           0
nummosquitos              0
wnvpresent                0
dtype: int64

#### Comments: 
- There is no missing data in train data.

In [15]:
train.describe()

,block,latitude,longitude,addressaccuracy,nummosquitos,wnvpresent
count,10506.000000,10506.000000,10506.000000,10506.000000,10506.000000,10506.000000
mean,35.687797,41.841139,-87.699908,7.819532,12.853512,0.052446
std,24.339468,0.112742,0.096514,1.452921,16.133816,0.222936
min,10.000000,41.644612,-87.930995,3.000000,1.000000,0.000000
25%,12.000000,41.732984,-87.760070,8.000000,2.000000,0.000000
50%,33.000000,41.846283,-87.694991,8.000000,5.000000,0.000000
75%,52.000000,41.954690,-87.627796,9.000000,17.000000,0.000000
max,98.000000,42.017430,-87.531635,9.000000,50.000000,1.000000


In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10506 entries, 0 to 10505
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   date                    10506 non-null  object 
 1   address                 10506 non-null  object 
 2   species                 10506 non-null  object 
 3   block                   10506 non-null  int64  
 4   street                  10506 non-null  object 
 5   trap                    10506 non-null  object 
 6   addressnumberandstreet  10506 non-null  object 
 7   latitude                10506 non-null  float64
 8   longitude               10506 non-null  float64
 9   addressaccuracy         10506 non-null  int64  
 10  nummosquitos            10506 non-null  int64  
 11  wnvpresent              10506 non-null  int64  
dtypes: float64(2), int64(4), object(6)
memory usage: 985.1+ KB


In [17]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116293 entries, 0 to 116292
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id                      116293 non-null  int64  
 1   date                    116293 non-null  object 
 2   address                 116293 non-null  object 
 3   species                 116293 non-null  object 
 4   block                   116293 non-null  int64  
 5   street                  116293 non-null  object 
 6   trap                    116293 non-null  object 
 7   addressnumberandstreet  116293 non-null  object 
 8   latitude                116293 non-null  float64
 9   longitude               116293 non-null  float64
 10  addressaccuracy         116293 non-null  int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 9.8+ MB


In [18]:
# convert to datetime
train["date"] = pd.to_datetime(train["date"], format='%Y-%m-%d')
test["date"] = pd.to_datetime(test["date"], format='%Y-%m-%d')

In [19]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10506 entries, 0 to 10505
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   date                    10506 non-null  datetime64[ns]
 1   address                 10506 non-null  object        
 2   species                 10506 non-null  object        
 3   block                   10506 non-null  int64         
 4   street                  10506 non-null  object        
 5   trap                    10506 non-null  object        
 6   addressnumberandstreet  10506 non-null  object        
 7   latitude                10506 non-null  float64       
 8   longitude               10506 non-null  float64       
 9   addressaccuracy         10506 non-null  int64         
 10  nummosquitos            10506 non-null  int64         
 11  wnvpresent              10506 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(4), ob

#### Traps Address check

In [20]:
trap_add = train[['trap','address']].groupby(by='trap').agg({'address':pd.Series.nunique})
trap_add[trap_add['address'] >1]

,address
trap,
T009,2
T035,2


In [21]:
train[train['trap']=='T009'].address.value_counts()

9100 West Higgins Road, Rosemont, IL 60018, USA     80
9100 West Higgins Avenue, Chicago, IL 60656, USA    31
Name: address, dtype: int64

In [22]:
train[train['trap']=='T035'].address.value_counts()

5100 West 72nd Street, Chicago, IL 60638, USA      45
3000 South Hoyne Avenue, Chicago, IL 60608, USA    27
Name: address, dtype: int64

#### Comments:

- T035 is probably mislabelled for one of the addresses and T009 is probably a typo. 
- Since coordinates are already present, trap numbers aren't important. 

#### Combine mosquito counts

As the test results are organized in a way that the number of mosquitoes are capped at 50 for each record
even when the samples are collected on the same day. We will combine these records into a single record.

In [23]:
train.shape

(10506, 12)

In [24]:
train= train.groupby([col for col in train.columns if col not in ['nummosquitos', \
                                                                'wnvpresent']]).sum()
train.reset_index(inplace=True)
train['wnvpresent'] = train['wnvpresent'].map(lambda x : 1 if x > 0 else x)

train.head(2)

,date,address,species,block,street,trap,addressnumberandstreet,latitude,longitude,addressaccuracy,nummosquitos,wnvpresent
0,2007-05-29,"1100 Roosevelt Road, Chicago, IL 60608, USA",CULEX PIPIENS/RESTUANS,11,W ROOSEVELT,T048,"1100 W ROOSEVELT, Chicago, IL",41.867108,-87.654224,8,1,0
1,2007-05-29,"1100 Roosevelt Road, Chicago, IL 60608, USA",CULEX RESTUANS,11,W ROOSEVELT,T048,"1100 W ROOSEVELT, Chicago, IL",41.867108,-87.654224,8,2,0


In [25]:
# split the date column into  year, month and day
train['year']=train.date.dt.year
train['month']=train.date.dt.month
train['day']=train.date.dt.day

test['year']=test.date.dt.year
test['month']=test.date.dt.month
test['day']=test.date.dt.day

In [26]:
train.head(2)

,date,address,species,block,street,trap,addressnumberandstreet,latitude,longitude,addressaccuracy,nummosquitos,wnvpresent,year,month,day
0,2007-05-29,"1100 Roosevelt Road, Chicago, IL 60608, USA",CULEX PIPIENS/RESTUANS,11,W ROOSEVELT,T048,"1100 W ROOSEVELT, Chicago, IL",41.867108,-87.654224,8,1,0,2007,5,29
1,2007-05-29,"1100 Roosevelt Road, Chicago, IL 60608, USA",CULEX RESTUANS,11,W ROOSEVELT,T048,"1100 W ROOSEVELT, Chicago, IL",41.867108,-87.654224,8,2,0,2007,5,29


In [27]:
train.shape

(8475, 15)

In [28]:
test.shape

(116293, 14)

### 3.2 Spray Data Inspection & Cleaning

In [29]:
spray.head(3)

,Date,Time,Latitude,Longitude
0,2011-08-29,6:56:58 PM,42.391623,-88.089163
1,2011-08-29,6:57:08 PM,42.391348,-88.089163
2,2011-08-29,6:57:18 PM,42.391022,-88.089157


In [30]:
#convert column names into lowercase
spray.columns=spray.columns.map(lambda x:x.lower())

In [31]:
spray.describe().T

,count,mean,std,min,25%,50%,75%,max
latitude,14835.0,41.904828,0.104381,41.713925,41.785001,41.940075,41.980978,42.395983
longitude,14835.0,-87.736690,0.067292,-88.096468,-87.794225,-87.727853,-87.694108,-87.586727


In [32]:
spray.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14835 entries, 0 to 14834
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       14835 non-null  object 
 1   time       14251 non-null  object 
 2   latitude   14835 non-null  float64
 3   longitude  14835 non-null  float64
dtypes: float64(2), object(2)
memory usage: 463.7+ KB


In [33]:
# convert to datetime
spray["date"] = pd.to_datetime(spray["date"],format='%Y-%m-%d')

In [34]:
spray.isnull().sum()

date           0
time         584
latitude       0
longitude      0
dtype: int64

#### Comments:
There are 584 rows with missing Time value and We have no clue to determine the true value, We will drop this column.

In [35]:
# drop "time" column from Spray data
spray.drop("time", axis=1, inplace=True)

In [36]:
spray.head()

,date,latitude,longitude
0,2011-08-29,42.391623,-88.089163
1,2011-08-29,42.391348,-88.089163
2,2011-08-29,42.391022,-88.089157
3,2011-08-29,42.390637,-88.089158
4,2011-08-29,42.390410,-88.088858


In [37]:
spray.shape

(14835, 3)

In [38]:
# check duplicates
spray[spray.duplicated()==True] 

,date,latitude,longitude
485,2011-09-07,41.983917,-87.793088
490,2011-09-07,41.986460,-87.794225
491,2011-09-07,41.986460,-87.794225
492,2011-09-07,41.986460,-87.794225
493,2011-09-07,41.986460,-87.794225
...,...,...,...
1025,2011-09-07,41.986460,-87.794225
1026,2011-09-07,41.986460,-87.794225
1027,2011-09-07,41.986460,-87.794225
1028,2011-09-07,41.986460,-87.794225


In [39]:
# there are 541 duplicated rows and We will remove them 
spray.drop_duplicates(keep='last', inplace = True)

In [40]:
spray.shape

(14294, 3)

#### Comments:
- After removing duplicates, We have 14294 records for spray data.

In [41]:
spray.head(2)

,date,latitude,longitude
0,2011-08-29,42.391623,-88.089163
1,2011-08-29,42.391348,-88.089163


In [42]:
spray.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14294 entries, 0 to 14834
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       14294 non-null  datetime64[ns]
 1   latitude   14294 non-null  float64       
 2   longitude  14294 non-null  float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 446.7 KB


In [43]:
train.date.dt.year.value_counts()

2007    2775
2013    2012
2009    1915
2011    1773
Name: date, dtype: int64

In [44]:
test.date.dt.year.value_counts()

2010    36557
2008    30498
2012    27115
2014    22123
Name: date, dtype: int64

In [45]:
spray.date.dt.year.value_counts()

2013    12626
2011     1668
Name: date, dtype: int64

#### Comments:
- Spray data has only year 2011 & 2013 data and only spray for 2 years records in train data.

In [46]:
# split the date column into  year, month and day
spray['year']=spray.date.dt.year
spray['month']=spray.date.dt.month
spray['day']=spray.date.dt.day

### 3.3 Weather Data Inspection & Cleaning

In [47]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Station      2944 non-null   int64  
 1   Date         2944 non-null   object 
 2   Tmax         2944 non-null   int64  
 3   Tmin         2944 non-null   int64  
 4   Tavg         2944 non-null   object 
 5   Depart       2944 non-null   object 
 6   DewPoint     2944 non-null   int64  
 7   WetBulb      2944 non-null   object 
 8   Heat         2944 non-null   object 
 9   Cool         2944 non-null   object 
 10  Sunrise      2944 non-null   object 
 11  Sunset       2944 non-null   object 
 12  CodeSum      2944 non-null   object 
 13  Depth        2944 non-null   object 
 14  Water1       2944 non-null   object 
 15  SnowFall     2944 non-null   object 
 16  PrecipTotal  2944 non-null   object 
 17  StnPressure  2944 non-null   object 
 18  SeaLevel     2944 non-null   object 
 19  Result

In [48]:
weather.columns=weather.columns.map(lambda x:x.lower())

In [49]:
# convert to datetime
weather["date"] = pd.to_datetime(weather["date"], format='%Y-%m-%d')

In [50]:
# split the date column into  year, month and day
weather['year']=weather.date.dt.year
weather['month']=weather.date.dt.month
weather['day']=weather.date.dt.day

In [51]:
weather.shape

(2944, 25)

In [52]:
weather.columns

Index(['station', 'date', 'tmax', 'tmin', 'tavg', 'depart', 'dewpoint',
       'wetbulb', 'heat', 'cool', 'sunrise', 'sunset', 'codesum', 'depth',
       'water1', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel',
       'resultspeed', 'resultdir', 'avgspeed', 'year', 'month', 'day'],
      dtype='object')

In [53]:
weather.describe()

,station,tmax,tmin,dewpoint,resultspeed,resultdir,year,month,day
count,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000
mean,1.500000,76.166101,57.810462,53.457880,6.960666,17.494905,2010.500000,7.500000,15.836957
std,0.500085,11.461970,10.381939,10.675181,3.587527,10.063609,2.291677,1.710236,8.855731
min,1.000000,41.000000,29.000000,22.000000,0.100000,1.000000,2007.000000,5.000000,1.000000
25%,1.000000,69.000000,50.000000,46.000000,4.300000,7.000000,2008.750000,6.000000,8.000000
50%,1.500000,78.000000,59.000000,54.000000,6.400000,19.000000,2010.500000,7.500000,16.000000
75%,2.000000,85.000000,66.000000,62.000000,9.200000,25.000000,2012.250000,9.000000,23.250000
max,2.000000,104.000000,83.000000,75.000000,24.100000,36.000000,2014.000000,10.000000,31.000000


In [54]:
weather.isnull().sum()

station        0
date           0
tmax           0
tmin           0
tavg           0
depart         0
dewpoint       0
wetbulb        0
heat           0
cool           0
sunrise        0
sunset         0
codesum        0
depth          0
water1         0
snowfall       0
preciptotal    0
stnpressure    0
sealevel       0
resultspeed    0
resultdir      0
avgspeed       0
year           0
month          0
day            0
dtype: int64

#### Comments:
- Seems there are no missing values from above checking, but some numeric columns data type are not correct. We will do further inspections for those columns.

In [55]:
weather.station.value_counts()

1    1472
2    1472
Name: station, dtype: int64

#### Comments:
- Equally sample data been collected from the 2 stations:
  - Station 1: CHICAGO O'HARE INTERNATIONAL AIRPORT Lat: 41.995 Lon: -87.933 Elev: 662 ft. above sea level
  - Station 2: CHICAGO MIDWAY INTL ARPT Lat: 41.786 Lon: -87.752 Elev: 612 ft. above sea level

In [56]:
weather.columns

Index(['station', 'date', 'tmax', 'tmin', 'tavg', 'depart', 'dewpoint',
       'wetbulb', 'heat', 'cool', 'sunrise', 'sunset', 'codesum', 'depth',
       'water1', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel',
       'resultspeed', 'resultdir', 'avgspeed', 'year', 'month', 'day'],
      dtype='object')

In [57]:
weather.tavg.unique()

array(['67', '68', '51', '52', '56', '58', 'M', '60', '59', '65', '70',
       '69', '71', '61', '55', '57', '73', '72', '53', '62', '63', '74',
       '75', '78', '76', '77', '66', '80', '64', '81', '82', '79', '85',
       '84', '83', '50', '49', '46', '48', '45', '54', '47', '44', '40',
       '41', '38', '39', '42', '37', '43', '86', '87', '89', '92', '88',
       '91', '93', '94', '90', '36'], dtype=object)

#### Comments:
- Missing values for tavg(average tempreture) as M

In [58]:
weather[weather.tavg=='M']

,station,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,year,month,day
7,2,2007-05-04,78,51,M,M,42,50,M,M,...,M,0.00,29.36,30.04,10.1,7,10.4,2007,5,4
505,2,2008-07-08,86,46,M,M,68,71,M,M,...,M,0.28,29.16,29.80,7.4,24,8.3,2008,7,8
675,2,2008-10-01,62,46,M,M,41,47,M,M,...,M,0.00,29.30,29.96,10.9,33,11.0,2008,10,1
1637,2,2011-07-22,100,71,M,M,70,74,M,M,...,M,0.14,29.23,29.86,3.8,10,8.2,2011,7,22
2067,2,2012-08-22,84,72,M,M,51,61,M,M,...,M,0.00,29.39,M,4.7,19,M,2012,8,22
2211,2,2013-05-02,71,42,M,M,39,45,M,M,...,M,0.00,29.51,30.17,15.8,2,16.1,2013,5,2
2501,2,2013-09-24,91,52,M,M,48,54,M,M,...,M,0.00,29.33,30.00,5.8,9,7.7,2013,9,24
2511,2,2013-09-29,84,53,M,M,48,54,M,M,...,M,0.22,29.36,30.01,6.3,36,7.8,2013,9,29
2525,2,2013-10-06,76,48,M,M,44,50,M,M,...,M,0.06,29.10,29.76,10.1,25,10.6,2013,10,6
2579,2,2014-05-02,80,47,M,M,43,47,M,M,...,M,0.04,29.10,29.79,10.7,23,11.9,2014,5,2


In [59]:
index = weather[weather.tavg=='M']
weather.drop(index.index, inplace= True)

#### Comments:

- There are 11 rows with missing values also more columns like 'depart', 'heat','cool','depth' etc. Seems these columns have common data input issues, I will remove these 11 rows and do a check again.

In [60]:
weather.depth.value_counts()

0    1472
M    1461
Name: depth, dtype: int64

In [61]:
weather.drop('depth', axis=1, inplace=True)

#### Comments:
- "depth" column has dropped as almost half of the values are missing and We don't have any clue to get its true value and not worth for data imputing.

In [62]:
weather.snowfall.value_counts()

M      1461
0.0    1459
  T      12
0.1       1
Name: snowfall, dtype: int64

In [63]:
weather.drop('snowfall', axis=1, inplace=True)

#### Comments:
- "snowfall" column has dropped as almost half of the values are missing and We don't have any clue to get its true value and not worth for data imputing.
- mosquitos are not likely to have in winter days, "snowfall" data column doesn't seem important to our analysis and modeling

In [64]:
weather.water1.value_counts()

M    2933
Name: water1, dtype: int64

In [65]:
weather.drop('water1', axis=1, inplace=True)

#### Comments:
- "water1" column has dropped as all records are not correct data.

In [66]:
weather[weather.codesum=='M']

,station,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,codesum,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,year,month,day


In [67]:
weather.preciptotal.unique()

array(['0.00', '  T', '0.13', '0.02', '0.38', '0.60', '0.14', '0.07',
       '0.11', '0.09', '1.01', '0.28', '0.04', '0.08', '0.01', '0.53',
       '0.19', '0.21', '0.32', '0.39', '0.31', '0.42', '0.27', '0.16',
       '0.58', '0.93', '0.05', '0.34', '0.15', '0.35', 'M', '0.40',
       '0.66', '0.30', '0.24', '0.43', '1.55', '0.92', '0.89', '0.17',
       '0.03', '1.43', '0.97', '0.26', '1.31', '0.06', '0.46', '0.29',
       '0.23', '0.41', '0.45', '0.83', '1.33', '0.91', '0.48', '0.37',
       '0.88', '2.35', '1.96', '0.20', '0.25', '0.18', '0.67', '0.36',
       '0.33', '1.28', '0.74', '0.76', '0.71', '0.95', '1.46', '0.12',
       '0.52', '0.64', '0.22', '1.24', '0.72', '0.73', '0.65', '1.61',
       '1.22', '0.50', '1.05', '2.43', '0.59', '2.90', '2.68', '1.23',
       '0.62', '6.64', '3.07', '1.44', '1.75', '0.82', '0.80', '0.86',
       '0.63', '0.55', '1.03', '0.70', '1.73', '1.38', '0.44', '1.14',
       '1.07', '3.97', '0.87', '0.78', '1.12', '0.68', '0.10', '0.61',
       '0.

In [68]:
weather[(weather.preciptotal=='M')|(weather.preciptotal=='T')]

,station,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,codesum,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,year,month,day
117,2,2007-06-28,73,61,67,M,56,61,0,2,...,,M,29.43,30.07,12.2,2,13.3,2007,6,28
119,2,2007-06-29,71,56,64,M,56,60,1,0,...,,M,29.47,30.11,7.4,2,8.2,2007,6,29


In [69]:
weather.drop(117,inplace=True)

In [70]:
weather.drop(118,inplace=True)

#### Comments:
- row 117 & 118 has removed as they have 2 missing values for "preciptotal".

In [71]:
weather.stnpressure.unique()

array(['29.10', '29.18', '29.38', '29.44', '29.39', '29.46', '29.31',
       '29.40', '29.57', '29.62', '29.29', '29.36', '29.21', '29.28',
       '29.20', '29.26', '29.33', '29.49', '29.54', '29.55', '29.23',
       '29.13', '29.19', '29.53', '29.60', '29.34', '29.41', '29.47',
       '29.51', '29.42', '29.43', '29.25', '29.03', '28.82', '28.87',
       '28.88', '29.16', '29.07', '28.84', '28.91', '29.24', 'M', '29.30',
       '29.12', '29.45', '29.56', '29.32', '29.05', '29.11', '29.06',
       '29.22', '29.08', '29.14', '29.37', '29.35', '29.15', '29.17',
       '29.48', '29.52', '29.27', '29.50', '28.59', '28.67', '28.75',
       '29.02', '29.79', '29.86', '29.63', '29.70', '28.95', '29.01',
       '28.79', '28.85', '28.97', '28.89', '28.94', '28.93', '28.98',
       '28.96', '29.00', '29.66', '29.09', '28.90', '29.04', '29.59',
       '29.65', '29.58', '29.61', '29.64', '29.71', '29.67', '28.80',
       '28.73', '29.68', '28.74', '28.55', '28.63', '28.92', '28.99',
       '28.81',

In [72]:
weather[(weather.stnpressure=='M')]

,station,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,codesum,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,year,month,day
87,2,2007-06-13,86,68,77,M,53,62,0,12,...,,0.00,M,M,7.0,5,M,2007,6,13
848,1,2009-06-26,86,69,78,7,60,M,0,13,...,,0.00,M,29.85,6.4,4,8.2,2009,6,26
2410,1,2013-08-10,81,64,73,0,57,M,0,8,...,,0.00,M,30.08,5.3,5,6.5,2013,8,10
2411,2,2013-08-10,81,68,75,M,55,63,0,10,...,,0.00,M,30.07,6.0,6,7.4,2013,8,10


In [73]:
index_names=weather[(weather.stnpressure=='M')]
weather.drop(index_names.index, inplace= True)

#### Comments:
- 4 missing value rows for "stnpressure" have been removed.

In [74]:
weather.depart.unique()

array(['14', 'M', '-3', ' 2', ' 4', ' 5', '10', '12', '13', '-2', '15',
       '11', '-4', '-6', ' 8', '-5', ' 1', ' 9', ' 6', '-9', '-8', ' 3',
       ' 0', '-1', '-7', ' 7', '-14', '18', '16', '22', '21', '20', '-10',
       '-16', '17', '-12', '-11', '-15', '-17', '19', '-13', '23'],
      dtype=object)

In [75]:
weather[(weather.depart=='M')]

,station,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,codesum,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,year,month,day
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,,0.00,29.18,29.82,2.7,25,9.6,2007,5,1
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,BR HZ,0.00,29.44,30.08,13.3,2,13.4,2007,5,2
5,2,2007-05-03,67,48,58,M,40,50,7,0,...,HZ,0.00,29.46,30.12,12.9,6,13.2,2007,5,3
9,2,2007-05-05,66,54,60,M,39,50,5,0,...,,T,29.46,30.09,11.2,7,11.5,2007,5,5
11,2,2007-05-06,68,52,60,M,30,46,5,0,...,,0.00,29.62,30.28,13.8,10,14.5,2007,5,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2935,2,2014-10-27,79,54,67,M,52,59,0,2,...,RA,0.02,29.00,29.67,12.7,19,13.6,2014,10,27
2937,2,2014-10-28,66,48,57,M,40,48,8,0,...,RA,0.03,29.23,29.85,14.0,26,14.6,2014,10,28
2939,2,2014-10-29,49,40,45,M,34,42,20,0,...,,0.00,29.42,30.07,8.5,29,9.0,2014,10,29
2941,2,2014-10-30,53,37,45,M,35,42,20,0,...,RA,T,29.41,30.10,5.9,23,6.5,2014,10,30


In [76]:
weather.drop('depart', axis=1, inplace=True)

#### Comments:

- There are almost half of "depart" data are missing. We have removed this column.

In [77]:
weather.sealevel.unique()

array(['29.82', '30.09', '30.08', '30.12', '30.05', '30.10', '30.29',
       '30.28', '30.03', '30.02', '29.94', '29.93', '29.92', '29.91',
       '30.04', '30.20', '30.19', '30.24', '29.97', '29.98', '29.84',
       '29.83', '30.27', '30.25', '30.26', '30.11', '30.06', '30.23',
       '30.15', '30.14', '30.00', '29.99', '29.90', '29.77', '29.76',
       '29.56', '29.54', '29.52', '29.51', '29.79', '29.78', '29.81',
       '29.55', '29.85', '30.07', '30.16', '29.96', '29.95', '30.13',
       '30.21', '30.22', '29.88', '30.01', '29.80', '29.89', '29.74',
       '29.87', '29.86', '30.18', '30.17', '29.34', '29.44', '29.45',
       '29.71', '29.72', '30.52', '30.53', '30.40', '30.41', '29.67',
       '29.53', '29.69', '29.61', '29.64', '29.63', '29.66', '29.70',
       '30.34', '30.33', '29.62', '29.60', '29.75', '29.68', '29.73',
       '30.31', '30.30', '30.32', '30.37', '30.39', 'M', '29.59', '29.65',
       '30.35', '30.36', '29.48', '30.38', '29.50', '29.25', '29.23',
       '29.46',

In [78]:
weather[(weather.sealevel=='M')]

,station,date,tmax,tmin,tavg,dewpoint,wetbulb,heat,cool,sunrise,...,codesum,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,year,month,day
832,1,2009-06-18,80,61,71,63,67,0,6,0416,...,RA BR,0.12,29.08,M,6.7,16,7.9,2009,6,18
994,1,2009-09-07,77,59,68,59,62,0,3,0523,...,BR,0.00,29.39,M,5.8,3,4.0,2009,9,7
1732,1,2011-09-08,75,57,66,53,59,0,1,0524,...,RA,T,29.34,M,13.0,2,13.4,2011,9,8
1745,2,2011-09-14,60,48,54,45,51,11,0,-,...,RA BR HZ FU,T,29.47,M,6.0,32,M,2011,9,14
1756,1,2011-09-20,74,49,62,54,58,3,0,0537,...,MIFG BCFG BR,0.00,29.26,M,7.3,18,7.3,2011,9,20
2090,1,2012-09-03,88,71,80,70,73,0,15,0519,...,BR,0.00,29.17,M,4.6,6,4.4,2012,9,3
2743,2,2014-07-23,76,64,70,56,61,0,5,-,...,,0.00,29.47,M,16.4,2,16.7,2014,7,23


In [79]:
index_names= weather[(weather.sealevel=='M')]
weather.drop(index_names.index, inplace= True)

#### Comments:

- There are 7 missing value rows for "sealevel", We have removed these rows from weather data.

In [80]:
weather[(weather.avgspeed=='M')]

,station,date,tmax,tmin,tavg,dewpoint,wetbulb,heat,cool,sunrise,...,codesum,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,year,month,day


In [81]:
weather.wetbulb.unique()

array(['56', '57', '47', '48', '50', '49', '46', '54', '53', '62', '63',
       '60', '61', '51', '59', '58', '43', '44', '45', '65', '66', '67',
       '64', '52', '55', '69', '68', '70', '72', '71', '73', '74', '76',
       '42', '39', '40', '41', '35', '33', '34', '75', '36', '32', '38',
       '78', '77', '37', 'M'], dtype=object)

In [82]:
weather[(weather.wetbulb=='M')]

,station,date,tmax,tmin,tavg,dewpoint,wetbulb,heat,cool,sunrise,...,codesum,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,year,month,day
2412,1,2013-08-11,81,60,71,61,M,0,6,0455,...,RA,0.01,29.35,30.07,2.0,27,3.0,2013,8,11
2415,2,2013-08-12,85,69,77,63,M,0,12,-,...,RA,0.66,29.27,29.92,4.5,26,7.7,2013,8,12


In [83]:
index_names= weather[(weather.wetbulb=='M')]
weather.drop(index_names.index, inplace= True)

#### Comments:

- There are 2 missing value rows for "wetbulb", We have removed these rows from weather data.

In [84]:
weather.reset_index(inplace=True)

In [85]:
weather.head()

,index,station,date,tmax,tmin,tavg,dewpoint,wetbulb,heat,cool,...,codesum,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,year,month,day
0,0,1,2007-05-01,83,50,67,51,56,0,2,...,,0.00,29.10,29.82,1.7,27,9.2,2007,5,1
1,1,2,2007-05-01,84,52,68,51,57,0,3,...,,0.00,29.18,29.82,2.7,25,9.6,2007,5,1
2,2,1,2007-05-02,59,42,51,42,47,14,0,...,BR,0.00,29.38,30.09,13.0,4,13.4,2007,5,2
3,3,2,2007-05-02,60,43,52,42,47,13,0,...,BR HZ,0.00,29.44,30.08,13.3,2,13.4,2007,5,2
4,4,1,2007-05-03,66,46,56,40,48,9,0,...,,0.00,29.39,30.12,11.7,7,11.9,2007,5,3


#### Merge train / test with Weather-station data

- In order to find the nearest weather station, we can use the coordinates of the Chicago O'Hare International Airport (where station 1 is located) and the Chicago Midway International Airport (where station 2 is located):

> - Station 1: CHICAGO O'HARE INTERNATIONAL AIRPORT Lat: 41.995 Lon: -87.933 Elev: 662 ft. above sea level
> - Station 2: CHICAGO MIDWAY INTL ARPT Lat: 41.786 Lon: -87.752 Elev: 612 ft. above sea level


In [86]:
# to see which weather monitoring station is close the the trap location
def nearest_station(lat, lon):
    station_1 = (41.995, -87.933)
    station_2 = (41.786, -87.752)
    stn1_dist = abs(geopy.distance.geodesic((lat,lon), station_1).km)
    stn2_dist = abs(geopy.distance.geodesic((lat,lon), station_2).km)
    return 1 if stn1_dist < stn2_dist else 2

In [87]:
train['nearest_station'] = [nearest_station(train.loc[i,'latitude'],train.loc[i,'longitude']) for i in train.index]
test['nearest_station'] = [nearest_station(test.loc[i,'latitude'],test.loc[i,'longitude']) for i in test.index]

In [88]:
train.head(2)

,date,address,species,block,street,trap,addressnumberandstreet,latitude,longitude,addressaccuracy,nummosquitos,wnvpresent,year,month,day,nearest_station
0,2007-05-29,"1100 Roosevelt Road, Chicago, IL 60608, USA",CULEX PIPIENS/RESTUANS,11,W ROOSEVELT,T048,"1100 W ROOSEVELT, Chicago, IL",41.867108,-87.654224,8,1,0,2007,5,29,2
1,2007-05-29,"1100 Roosevelt Road, Chicago, IL 60608, USA",CULEX RESTUANS,11,W ROOSEVELT,T048,"1100 W ROOSEVELT, Chicago, IL",41.867108,-87.654224,8,2,0,2007,5,29,2


In [89]:
train['nearest_station'].value_counts(normalize= True)

2    0.709381
1    0.290619
Name: nearest_station, dtype: float64

In [90]:
test['nearest_station'].value_counts(normalize= True)

2    0.695588
1    0.304412
Name: nearest_station, dtype: float64

#### Comments:
- Both train & test data, more traps close to weather monitoring station 2. (CHICAGO MIDWAY INTL ARPT Lat: 41.786 Lon: -87.752 Elev: 612 ft. above sea level)

In [91]:
train.shape

(8475, 16)

In [92]:
test.shape

(116293, 15)

## 4. Export Cleaned Data

In [93]:
train.to_csv('../assets/train_cleaned.csv', index=False)
test.to_csv('../assets/test_cleaned.csv', index=False)
weather.to_csv('../assets/weather_cleaned.csv', index=False)
spray.to_csv('../assets/spray_cleaned.csv', index=False)

Cleaned Datasets will be used for Part 2- EDA and Part 3-Modeling.